In [ ]:
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123')
sys.path.append('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/training')
sys.path.append('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/preprocessing/')
from layers import nms,iou

%matplotlib inline

In [ ]:
"""

double check that the process for creating the clan data is the same for
dicom and mhd; this verifies the process before moving to the cluster

"""
dic_clean = np.load('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/prep_result/dicom/summit-2264-sze_Y0_BASELINE_A_clean.npy')
mhd_clean = np.load('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/prep_result/mhd/summit-2264-sze_Y0_BASELINE_A_clean.npy')

dic_lbl = np.load('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/bbox_result/dicom/summit-2264-sze_Y0_BASELINE_A_lbb.npy')
mhd_lbl = np.load('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/bbox_result/mhd/summit-2264-sze_Y0_BASELINE_A_lbb.npy')

dic_pbb = np.load('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/bbox_result/dicom/summit-2264-sze_Y0_BASELINE_A_pbb.npy')
mhd_pbb = np.load('/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/bbox_result/mhd/summit-2264-sze_Y0_BASELINE_A_pbb.npy')

print('Do the _clean numpy arrays from mhd and dicom match?', np.array_equal(dic_clean, mhd_clean))
print('Do the _label numpy arrays from mhd and dicom match?', np.array_equal(dic_lbl, mhd_lbl))
print('Do the _pbb numpy arrays from mhd and dicom match?', np.array_equal(dic_pbb, mhd_pbb))

In [ ]:
# find a scan that has been downloaded
import os

BBOX_PATH = '/Users/john/Projects/SOTAEvaluationNoduleDetection/models/grt123/bbox_result'
list_test_scans = [
    os.path.join(BBOX_PATH, lbb) 
    for lbb in os.listdir(BBOX_PATH) 
    if lbb.endswith('_lbb.npy') and \
        lbb.split('_',1)[0] in os.listdir('/Users/john/Projects/SOTAEvaluationNoduleDetection/scans/lung50')
    ]


list_test_scans

In [ ]:
"""
check the lbb and pbb pulled from cluster to check that things are consistent with
what was produced locally

"""
from utils import *

stem = 'summit-7895-qcw'

lbl = np.load(f'{BBOX_PATH}/{stem}_Y0_BASELINE_A_lbb.npy')
pbb = np.load(f'{BBOX_PATH}/{stem}_Y0_BASELINE_A_pbb.npy')
pbb = nms(pbb,0.05)
pbb = pbb[pbb[:,0]>-1]

lbl_local = np.load(f'{BBOX_PATH}/mhd/{stem}_Y0_BASELINE_A_lbb.npy')
pbb_local = np.load(f'{BBOX_PATH}/mhd/{stem}_Y0_BASELINE_A_pbb.npy')
pbb_local = nms(pbb_local,0.05)
pbb_local = pbb_local[pbb_local[:,0]>-1]

print(np.array_equal(lbl_local, lbl))
print(np.array_equal(np.int16(pbb_local), np.int16(pbb)))